In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Check Fold


> ## 📌 ใช้ค่า MCV, MCH, Hb เทรน ML 
> ## 📌 ใช้ภาพ 2dFFT image 3 channels 

- 🩸แบบ 2 โรค (E, M)


### 📂 DATA : ค่าจากผล Lab

In [2]:
# Read excel file with sheet name
dict_df = pd.read_excel('/home/kannika/code/Rheology2023/Rheology_Blood/All lab results (Blood viscosity work)3.xlsx', 
                   sheet_name=['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม', 'Normal'], header=2)
dict_df

{'Patients_ตัดม้าม':         No.  Code         HN        AGE          PC  PS activity    Free PS  \
 0         1  HN01  2045512.0  29.000000   76.800000    43.000000  58.600000   
 1         2  HN02  4045280.0  21.000000  100.400000    58.800000  70.600000   
 2         3  HN03  3024257.0  41.000000   75.600000    52.500000  57.900000   
 3         4  HN04  1257101.0  40.000000   31.700000    57.700000  59.300000   
 4         6  HN06  1797136.0  32.000000   43.700000    41.100000  77.300000   
 5         7  HN07  2234214.0  50.000000   66.400000    54.500000  62.100000   
 6        12  HN12  1110168.0  45.000000   87.800000    39.300000  60.000000   
 7        13  HN13  3949905.0  23.000000   50.400000    37.000000  83.600000   
 8        16  HN16  1665191.0  35.000000   79.000000    47.400000  59.100000   
 9        17  HN17  2272494.0  27.000000   47.600000    34.200000  59.600000   
 10       19  HN19  1513782.0  36.000000   55.500000    83.000000  76.600000   
 11       23  HN23  

In [3]:
''''
subclass
Patients_ตัดม้าม: Splenectomy
Patients_ไม่ตัดม้าม: No_Splenectomy
Normal
'''

dict_sheet =  ['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม']
lstdict_code  = []
for sheet in dict_sheet:
    df_sheet = dict_df[sheet]
    df_sheet["sheet"] = sheet
    dict_code = df_sheet[['Code','MCV', 'MCH', 'Hb', 'typeBEvsBM', 'sheet']]
    #dict_code = dict_code[dict_code['Code'].notna()]
    _dict_code = dict_code[(dict_code['Code'].notna())&(dict_code['MCV'].notna())&(dict_code['MCH'].notna())
                         &dict_code['Hb'].notna()]
    __dict_code = _dict_code[(_dict_code['MCV']!= '-')|(_dict_code['MCH']!= '-')
                         |(_dict_code['Hb']!= '-')]
    lstdict_code.append(__dict_code)
    print(dict_df[sheet].shape)

print(len(lstdict_code))
df_dictCode = pd.concat(lstdict_code, ignore_index=True)
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ตัดม้าม','Splenectomy')
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ไม่ตัดม้าม','No_Splenectomy')
print(df_dictCode.shape)
df_dictCode.head()

(27, 52)
(27, 52)
2
(44, 6)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet
0,HN01,75.8,23.1,6.0,E,Splenectomy
1,HN02,73.5,22.4,7.2,E,Splenectomy
2,HN03,77.3,24.3,9.0,E,Splenectomy
3,HN04,79.8,25.6,7.6,E,Splenectomy
4,HN06,78.5,25.9,8.2,E,Splenectomy


In [4]:
print(len(list(set(df_dictCode["Code"]))))
print(set(df_dictCode["sheet"]))
print(set(df_dictCode["typeBEvsBM"]))

44
{'Splenectomy', 'No_Splenectomy'}
{'E', 'M', 'AEBart'}


In [5]:
df_dictCode.isnull().values.any()

False

In [6]:
#df_dictCode.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_typeBEvsBM.csv")

In [98]:
EM_list = list(set(df_dictCode["typeBEvsBM"]))
EM_list.sort()
for j in range(len(EM_list)):
    classes_j = EM_list[j]
    df_j = df_dictCode[df_dictCode["typeBEvsBM"]==classes_j]
    print(f"[INFO]: Classes: {classes_j} ==> {df_j.shape[0]}")

[INFO]: Classes: AEBart ==> 2
[INFO]: Classes: E ==> 31
[INFO]: Classes: M ==> 11


> ### Map clasess "E, M"

In [99]:
DF_B_res_ML= pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
del DF_B_res_ML["fold"]
print(DF_B_res_ML.shape)
DF_B_res_ML.head()

(51, 8)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4
3,3,NBL14,NBL,Normal,0.0,86.8,28.1,13.2
4,4,NBL23,NBL,Normal,0.0,66.5,20.3,13.0


In [100]:
## 1. Create Dict typeBEvsBM
df_dictCode_ = df_dictCode[df_dictCode["typeBEvsBM"]!='AEBart'].reset_index(drop=True)
lst_Code = df_dictCode_["Code"].tolist()
lst_EM = df_dictCode_["typeBEvsBM"].tolist()
dict_EM = zip(lst_Code, lst_EM)
output_EM = dict(dict_EM)
print(output_EM)

{'HN01': 'E', 'HN02': 'E', 'HN03': 'E', 'HN04': 'E', 'HN06': 'E', 'HN07': 'E', 'HN12': 'E', 'HN13': 'E', 'HN16': 'E', 'HN17': 'E', 'HN19': 'M', 'HN23': 'E', 'HN24': 'M', 'HN25': 'M', 'HN27': 'E', 'HN28': 'M', 'HN32': 'E', 'HN36': 'M', 'HN39': 'E', 'HN40': 'M', 'HN44': 'E', 'HN05': 'E', 'HN08': 'E', 'HN09': 'E', 'HN10': 'M', 'HN11': 'M', 'HN14': 'E', 'HN18': 'E', 'HN20': 'E', 'HN21': 'E', 'HN22': 'M', 'HN26': 'E', 'HN29': 'E', 'HN30': 'E', 'HN31': 'E', 'HN33': 'E', 'HN34': 'E', 'HN35': 'E', 'HN38': 'E', 'HN41': 'M', 'HN42': 'E', 'HN43': 'M'}


In [101]:
DF_B_res_ML["typeBEvsBM"]= DF_B_res_ML['Code'].map(output_EM)
DFML_EM = DF_B_res_ML[DF_B_res_ML['typeBEvsBM'].notna()]
print(set(DFML_EM["typeBEvsBM"]))
print(DFML_EM.shape)
DFML_EM.head()

{'E', 'M'}
(33, 9)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1,E
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0,E
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4,M
6,6,HN17,HN,Splenectomy,1.0,75.0,25.0,7.1,E
7,7,HN12,HN,Splenectomy,1.0,75.4,24.1,9.8,E


In [102]:
EM_list = list(set(DFML_EM["typeBEvsBM"]))
EM_list.sort()
for j in range(len(EM_list)):
    classes_j = EM_list[j]
    df_j = DFML_EM[DFML_EM["typeBEvsBM"]==classes_j]
    print(f"[INFO]: Classes: {classes_j} ==> {df_j.shape[0]}")

[INFO]: Classes: E ==> 24
[INFO]: Classes: M ==> 9


In [15]:
#DFML_EM.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_typeBEvsBM_Consist_2dFFTdata.csv")

In [103]:
list_code_2dfft = list(set(DFML_EM["Code"]))
list_code_2dfft.sort()
print("Number of Cases: ", len(list_code_2dfft))

Number of Cases:  33


> #### 💾 Split 3-Fold

- E:M = 8:3

In [104]:
import random

EM_list = list(set(DFML_EM["typeBEvsBM"]))
EM_list.sort()
DF_fold = pd.DataFrame(columns=list(DFML_EM.columns))

for j in range(len(EM_list)):
    classes_n = EM_list[j]
    if classes_n == "E":
        n=8
    elif classes_n == "M":
         n=3
    df_classes = DFML_EM[DFML_EM["typeBEvsBM"]==classes_n]
    print(f"[INFO]: [{j+1}].  For Classes {classes_n} With Shape ==> {df_classes.shape}")
    for i in range(1, 4):
        df_classes_ran = df_classes.sample(n, replace=False)
        df_classes.drop(index=df_classes_ran.index, inplace=True)
        df_classes_ran["fold"] = i
        DF_fold = DF_fold.append(df_classes_ran)
        if i == 3:
            #print(df_classes.shape[0])
            break
    df_classes["fold"] = 3
    DF_fold = DF_fold.append(df_classes)

[INFO]: [1].  For Classes E With Shape ==> (24, 9)
[INFO]: [2].  For Classes M With Shape ==> (9, 9)


/tmp/ipykernel_3651/2378620871.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_3651/2378620871.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_fold = DF_fold.append(df_classes_ran)
/tmp/ipykernel_3651/2378620871.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_3651/2378620871.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [105]:
DF_fold = DF_fold.reset_index(drop=True)
print(DF_fold.shape)
DF_fold['fold'] = DF_fold['fold'].astype(int)
DF_fold.head()

(33, 10)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM,fold
0,42,HN44,HN,Splenectomy,1.0,61.7,19.7,7.0,E,1
1,17,HN21,HN,No_Splenectomy,1.0,64.8,19.7,6.5,E,1
2,8,HN13,HN,Splenectomy,1.0,72.6,23.4,5.8,E,1
3,43,HN29,HN,No_Splenectomy,1.0,64.2,19.1,6.6,E,1
4,50,HN39,HN,Splenectomy,1.0,76.4,22.5,6.4,E,1


In [106]:
### Check fold
EM_list = list(set(DF_fold["typeBEvsBM"]))
EM_list.sort()
fold_list = list(set(DF_fold["fold"]))
fold_list.sort()
for j in range(len(fold_list)):
    fold_j = fold_list[j]
    df_fold_j = DF_fold[DF_fold["fold"]==fold_j]
    print(f"[INFO]: Fold-{fold_j}: {df_fold_j.shape[0]} Cases")
    for k in range(len(EM_list)):
        classes_k = EM_list[k]
        df_k = df_fold_j[df_fold_j["typeBEvsBM"]==classes_k]
        print(f"[INFO]: Classes: {classes_k} ==> {df_k.shape[0]}")
    print("="*125)

[INFO]: Fold-1: 11 Cases
[INFO]: Classes: E ==> 8
[INFO]: Classes: M ==> 3
[INFO]: Fold-2: 11 Cases
[INFO]: Classes: E ==> 8
[INFO]: Classes: M ==> 3
[INFO]: Fold-3: 11 Cases
[INFO]: Classes: E ==> 8
[INFO]: Classes: M ==> 3


In [111]:
DF_fold.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_EMclass.csv")

> #### Defind Fold 

In [112]:
## 1. Create Dict fold
lst_Code = DF_fold["Code"].tolist()
lst_fold = DF_fold["fold"].tolist()
dict_fold = zip(lst_Code, lst_fold)
output_fold = dict(dict_fold)
print(output_fold)

{'HN44': 1, 'HN21': 1, 'HN13': 1, 'HN29': 1, 'HN39': 1, 'HN16': 1, 'HN18': 1, 'HN34': 1, 'HN35': 2, 'HN02': 2, 'HN26': 2, 'HN42': 2, 'HN01': 2, 'HN30': 2, 'HN33': 2, 'HN17': 2, 'HN20': 3, 'HN27': 3, 'HN31': 3, 'HN38': 3, 'HN23': 3, 'HN12': 3, 'HN14': 3, 'HN32': 3, 'HN24': 1, 'HN19': 1, 'HN41': 1, 'HN25': 2, 'HN28': 2, 'HN40': 2, 'HN43': 3, 'HN22': 3, 'HN36': 3}


In [113]:
## 2. Create Dict typeBEvsBM
lst_EM = DF_fold["typeBEvsBM"].tolist()
dict_EM = zip(lst_Code, lst_EM)
output_EM = dict(dict_EM)
print(output_EM)

{'HN44': 'E', 'HN21': 'E', 'HN13': 'E', 'HN29': 'E', 'HN39': 'E', 'HN16': 'E', 'HN18': 'E', 'HN34': 'E', 'HN35': 'E', 'HN02': 'E', 'HN26': 'E', 'HN42': 'E', 'HN01': 'E', 'HN30': 'E', 'HN33': 'E', 'HN17': 'E', 'HN20': 'E', 'HN27': 'E', 'HN31': 'E', 'HN38': 'E', 'HN23': 'E', 'HN12': 'E', 'HN14': 'E', 'HN32': 'E', 'HN24': 'M', 'HN19': 'M', 'HN41': 'M', 'HN25': 'M', 'HN28': 'M', 'HN40': 'M', 'HN43': 'M', 'HN22': 'M', 'HN36': 'M'}


In [114]:
DF_2dFFTOrigin = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold.csv")
del DF_2dFFTOrigin["fold"]
print(DF_2dFFTOrigin.shape)
DF_2dFFTOrigin_ = DF_2dFFTOrigin[DF_2dFFTOrigin['subclass']!='Normal']
print("Classes: Not Normal ==> ", DF_2dFFTOrigin_.shape)
DF_2dFFTOrigin_.head()

(538, 11)
Classes: Not Normal ==>  (337, 11)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1


In [115]:
##**3. Map typeBEvsBM
DF_2dFFTOrigin_["typeBEvsBM"]= DF_2dFFTOrigin_['Code'].map(output_EM)
print(set(DF_2dFFTOrigin_["typeBEvsBM"]))
DF_2dFFTOrigin_.head()

{nan, 'E', 'M'}


/tmp/ipykernel_3651/3304599483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dFFTOrigin_["typeBEvsBM"]= DF_2dFFTOrigin_['Code'].map(output_EM)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E


In [116]:
DF_2dFFTEM = DF_2dFFTOrigin_[DF_2dFFTOrigin_['typeBEvsBM'].notna()]
print(set(DF_2dFFTEM["typeBEvsBM"]))
print(DF_2dFFTEM.shape)
DF_2dFFTEM.head()

{'E', 'M'}
(317, 12)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E


In [117]:
##**4. Map Fold
DF_2dFFTEM["fold"]= DF_2dFFTEM['Code'].map(output_fold)
print(set(DF_2dFFTEM["fold"]))
DF_2dFFTEM.head()

{1, 2, 3}


/tmp/ipykernel_3651/2223037834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dFFTEM["fold"]= DF_2dFFTEM['Code'].map(output_fold)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM,fold
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E,2
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E,2
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E,2
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E,2
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E,2


In [118]:
print(DF_2dFFTEM.shape)
DF_2dFFTEM.isnull().values.any()

(317, 13)


False

In [119]:
list_code_2dfft = list(set(DF_2dFFTEM["Code"]))
list_code_2dfft.sort()
print("Number of Cases: ", len(list_code_2dfft))

Number of Cases:  33


In [120]:
set(lst_Code) - set(list_code_2dfft)

set()

In [121]:
set(list_code_2dfft) - set(lst_Code)

set()

In [122]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 3-Fold With Class:")
        lst_cls = list(set(df_classes['typeBEvsBM']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["typeBEvsBM"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    return

In [123]:
_(DF_2dFFTEM)

[INFO]: [1].  Fold 1 With Shape ==> 96 Folder, (96, 13)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 68 Folder, 68 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 114 Folder, (114, 13)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 86 Folder, 86 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 107 Folder, (107, 13)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 75 Folder, 75 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 32 Folder, 32 images!! ==> 3 PerSon!!


In [124]:
lst_fold = list(set(DF_2dFFTEM["fold"]))
lst_fold.sort()
len_EM = list()
for j in range(len(lst_fold)):
    df_fold = DF_2dFFTEM[DF_2dFFTEM["fold"]==lst_fold[j]]
    lst_cls = list(set(df_fold['typeBEvsBM']))
    lst_cls.sort()
    for m in lst_cls:
        df_f =  df_fold[df_fold["typeBEvsBM"]==m]
        len_EM.append(df_f.shape[0])

In [125]:
print(len_EM)
print(min(len_EM))

[68, 28, 86, 28, 75, 32]
28


In [188]:
# def Splitfold(classes, DF_2dFFTEM):
#     len_EM=28
#     if classes == "E":
#         n = 3 ## Classes E
#     else:
#         nM = 8 # Classes M
#     Dataset_all = list()
#     for j in range(len(lst_fold)):
#         #Dataset_fold = pd.DataFrame(columns=list(DF_2dFFTEM.columns))
#         Dataset_foldE = pd.DataFrame(columns=list(DF_2dFFTEM.columns))
#         print(f"[INFO]: Fold-{lst_fold[j]}")
#         df_fold = DF_2dFFTEM[DF_2dFFTEM["fold"]==lst_fold[j]]
#         df_E =  df_fold[df_fold["typeBEvsBM"]==classes]
#         code_lstE = list(set(df_E["Code"]))
#         code_lstE.sort()
#         for n in range(len(code_lstE)):
#             df_E_code = df_E[df_E["Code"]==code_lstE[n]]
#             df_E_code_ran = df_E_code.sample(n, replace=False)
#             Dataset_foldE = Dataset_foldE.append(df_E_code_ran)
#             df_E.drop(index=df_E_code_ran.index, inplace=True)       
#         df_E_code_ran = df_E.sample(len_EM-len(Dataset_foldE), replace=False)
#         Dataset_foldE = Dataset_foldE.append(df_E_code_ran)
#         Dataset_all.append(Dataset_foldE)
        
#     result = pd.concat(Dataset_all, ignore_index=True)
    
#     return result

In [189]:
# ## 1. Classes ==> BE 
# classes = "E"
# result_E = Splitfold(classes, DF_2dFFTEM)
# print(result_E.shape)
# print(set(result_E["fold"]))
# print(set(result_E["typeBEvsBM"]))
# print(len(list(set(result_E["Code"]))), "Cases")
# result_E.head()

In [190]:
# ## 2. Classes ==> BM
# classes = "M"
# result_M = Splitfold(classes, DF_2dFFTEM)
# print(result_M.shape)
# print(set(result_M["fold"]))
# print(set(result_M["typeBEvsBM"]))
# print(len(list(set(result_M["Code"]))), "Cases")
# result_E.head()

In [191]:
# result = pd.concat([result_E, result_M], ignore_index=True)
# print(result.shape)
# result.head()

In [192]:
# _(result)

In [184]:
Dataset = pd.DataFrame(columns=list(DF_2dFFTEM.columns))
for j in range(len(lst_fold)):
    df_fold_m = DF_2dFFTEM[DF_2dFFTEM["fold"]==lst_fold[j]]
    lst_cls = list(set(df_fold['typeBEvsBM']))
    lst_cls.sort()
    for m in lst_cls:
        df_f =  df_fold_m[df_fold_m["typeBEvsBM"]==m]
        df_classes_ran = df_f.sample(min(len_EM), replace=False)
        #df_classes.drop(index=df_classes_ran.index, inplace=True)
        Dataset = Dataset.append(df_classes_ran)

/tmp/ipykernel_3651/503871875.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataset = Dataset.append(df_classes_ran)
/tmp/ipykernel_3651/503871875.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataset = Dataset.append(df_classes_ran)
/tmp/ipykernel_3651/503871875.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataset = Dataset.append(df_classes_ran)
/tmp/ipykernel_3651/503871875.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dataset = Dataset.append(df_classes_ran)
/tmp/ipykernel_3651/503871875.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p

In [185]:
print(Dataset.shape)
print(set(Dataset["fold"]))
Dataset.head()

(168, 13)
{1, 2, 3}


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM,fold
185,185,185,HN29_D0_30HZ_20XINF_UWELL_25660117_150029_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN29_D0_30HZ_20XINF_UWELL_25660117_150029,SSD_Backup,HN,HN29,No_Splenectomy,"[2, 5, 7]",1,E,1
278,278,278,HN39_D0_30HZ_20XINF_UWELL_25660223_151435_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_151435,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1
280,280,280,HN39_D0_30HZ_20XINF_UWELL_25660223_151842_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_151842,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1
286,286,286,HN39_D0_30HZ_20XINF_UWELL_25660223_153102_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_153102,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1
48,48,48,HN13_D0_30HZ_20XINF_UWELL_25651124_144313_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN13_D0_30HZ_20XINF_UWELL_25651124_144313,SSD_Backup,HN,HN13,Splenectomy,"[2, 5, 7]",1,E,1


In [194]:
Dataset = Dataset.reset_index(drop=True)
typeBEvsBM_binary  = list()
for i in range(len(Dataset)):
    if Dataset["typeBEvsBM"][i] == "E" : 
        typeBEvsBM_binary.append(0)
    else:
        typeBEvsBM_binary.append(1)
print(len(typeBEvsBM_binary))
typeBEvsBM_binary

168


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [195]:
Dataset["typeBEvsBM_binary"]=typeBEvsBM_binary
print(Dataset.shape)
Dataset.head()

(168, 14)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM,fold,typeBEvsBM_binary
0,185,185,HN29_D0_30HZ_20XINF_UWELL_25660117_150029_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN29_D0_30HZ_20XINF_UWELL_25660117_150029,SSD_Backup,HN,HN29,No_Splenectomy,"[2, 5, 7]",1,E,1,0
1,278,278,HN39_D0_30HZ_20XINF_UWELL_25660223_151435_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_151435,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
2,280,280,HN39_D0_30HZ_20XINF_UWELL_25660223_151842_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_151842,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
3,286,286,HN39_D0_30HZ_20XINF_UWELL_25660223_153102_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN39_D0_30HZ_20XINF_UWELL_25660223_153102,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
4,48,48,HN13_D0_30HZ_20XINF_UWELL_25651124_144313_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN13_D0_30HZ_20XINF_UWELL_25651124_144313,SSD_Backup,HN,HN13,Splenectomy,"[2, 5, 7]",1,E,1,0


In [196]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0.1       168 non-null    object
 1   Unnamed: 0         168 non-null    object
 2   image_name         168 non-null    object
 3   image_path         168 non-null    object
 4   image_folder       168 non-null    object
 5   folder_source      168 non-null    object
 6   classes            168 non-null    object
 7   Code               168 non-null    object
 8   subclass           168 non-null    object
 9   dt                 168 non-null    object
 10  classes_binary     168 non-null    object
 11  typeBEvsBM         168 non-null    object
 12  fold               168 non-null    object
 13  typeBEvsBM_binary  168 non-null    int64 
dtypes: int64(1), object(13)
memory usage: 18.5+ KB


In [198]:
_(Dataset)

[INFO]: [1].  Fold 1 With Shape ==> 56 Folder, (56, 14)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 56 Folder, (56, 14)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 56 Folder, (56, 14)
Check 3-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!


In [199]:
Dataset.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Blood_Viscosity-2dFFTdataset-3channels-3Fold-EMClasses.csv")